In [0]:
import re
import csv
import string
from collections import defaultdict
from tqdm import tnrange, tqdm_notebook
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.nn.utils.rnn import pad_sequence
import nltk
from razdel import sentenize
from nltk.corpus import stopwords
import string
from nltk.tokenize import wordpunct_tokenize as tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from nltk.stem.snowball import SnowballStemmer

In [169]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
SEED = 1
torch.manual_seed(SEED)
np.random.seed(SEED)

In [171]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [174]:
data = []
with open('train.csv', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
    isfirst = True
    for row in tqdm(csv_reader):
        if isfirst:
            isfirst = False
            continue
        sentences = nltk.sent_tokenize(row[2])
        question = row[3]
        if question.endswith('?'):
            question = question[:-1]
        answer = row[4]
        if answer.endswith("..."):
            answer = answer[:-3]
        if answer.startswith("..."):
            answer = answer[3:]
        if answer.startswith(' '):
            answer = answer[1:]
        if answer[0] in string.punctuation:
            answer = answer[1:]
        if answer[-1] in string.punctuation:
            answer = answer[:-1]
        answer = answer.strip()
        answer_sentence, answer_index = None, None
        for sentence in sentences:
            index = sentence.lower().find(answer.lower())
            if index != -1:
                answer_sentence, answer_index = sentence, index
                break
        if answer_index:
            answer_sentence = re.split('(\W)', answer_sentence)
            question = re.split('(\W)', question)
            data.append((answer_sentence, question, answer_index, len(answer)))

In [0]:
stemmer = SnowballStemmer("russian")

In [0]:
word_to_index = defaultdict(lambda: 0)
separator = 65000
pad_value = 65001

In [0]:
def stem_words(sentence, query):
    index = []

    for word in sentence:
        stemmed_word = stemmer.stem(word.lower())

        if stemmed_word not in word_to_index:
            word_to_index[stemmed_word] = len(word_to_index) + 1
            
        index.append(word_to_index[stemmed_word])
    index.append(separator)

    for word in query:
        stemmed_word = stemmer.stem(word.lower())

        if stemmed_word not in word_to_index:
            word_to_index[stemmed_word] = len(word_to_index) + 1
        
        index.append(word_to_index[stemmed_word])

    return torch.tensor(index)

In [0]:
def get_words_indices(sentence):
    words_indices = []
    index = 0
    for word in sentence:
        words_indices.append((index, len(word)))
        index += len(word)
    return words_indices

In [177]:
stemmed_dataset = [stem_words(item[0], item[1]) for item in tqdm(data)]
stemmed_padded_data = pad_sequence(stemmed_dataset, batch_first=True, padding_value=pad_value)

In [178]:
indices_data = [get_words_indices(item[0]) for item in tqdm(data)]

In [0]:
def join_dataset(stemmed_padded_data, indices_data, data):
    dataset_joined = []

    for sentence, indices, elem in tqdm(zip(stemmed_padded_data, indices_data, data)):
        answer_start, answer_len = elem[2], elem[3]
        y = np.array([0, 0])

        for index in range(len(indices)):
            if indices[index][0] == answer_start:
                y[0] = index
            if indices[index][0] == answer_start + answer_len:
                y[1] = index - 1
            
        dataset_joined.append(np.append(sentence.numpy(), y))

    return dataset_joined

In [180]:
data_joined = join_dataset(stemmed_padded_data, indices_data, data)

In [0]:
train, val = train_test_split(data_joined, test_size=0.2, random_state=42)

In [0]:
class LSTMModel(nn.Module):
    def __init__(self, num_layers, vocab_size):
        super(LSTMModel, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, 64)
        self.lstm = nn.LSTM(64, 64, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.fc2 = nn.Linear(hidden_dim * 2, 2)
    
    def forward(self, x):
        x = self.word_embeddings(x)
        x, _ = self.lstm(x)
        x = self.fc2(x)
        x = torch.transpose(x, 1, 2)
        x = F.log_softmax(x, dim=2)
        
        return x

In [0]:
def fit(model, train, val, optimizer, loss_function, epoch_cnt, batch_size):
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size)
    train_loss_values = []
    val_loss_values = []
            
    for epoch in range(epoch_cnt):
        for batch_data in train_loader:
            x, y = batch_data[:, :-2].to(device), batch_data[:, -2:].to(device)
            optimizer.zero_grad()
            output = model(x.long())
            loss = (loss_function(output[:, 0], y[:, 0].reshape(-1).long()) +\
                    loss_function(output[:, 1], y[:, 1].reshape(-1).long())) / 2
            loss.backward()

            train_loss_values.append(loss.item())
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()

        with torch.no_grad():
            loss_values = []
            for batch_data in val_loader:
                x, y = batch_data[:, :-2].to(device), batch_data[:, -2:].to(device)
                output = model(x.long())
                loss = (loss_function(output[:, 0], y[:, 0].reshape(-1).long()) +\
                        loss_function(output[:, 1], y[:, 1].reshape(-1).long())) / 2

                loss_values.append(loss.item())
            val_loss_values.append(np.mean(np.array(loss_values)))

        print("Epoch " + str(epoch) + " done.")
    return train_loss_values, val_loss_values 


In [0]:
num_layers = 4
vocab_size = max(pad_value, len(word_to_index)) + 1
epoch_cnt = 10
batch_size = 128

In [0]:
torch.cuda.empty_cache()

In [0]:
model = LSTMModel(num_layers, vocab_size)
model = model.float()
model = model.to(device)

In [0]:
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [188]:
train_loss_values, val_loss_values =\
    fit(model, train, val, optimizer, loss_function, epoch_cnt, batch_size)

Epoch 0 done.
Epoch 1 done.
Epoch 2 done.
Epoch 3 done.
Epoch 4 done.
Epoch 5 done.
Epoch 6 done.
Epoch 7 done.
Epoch 8 done.
Epoch 9 done.


In [189]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [205]:
pip install pymorphy2

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 13.5MB/s 


In [0]:
def find_closest_sentence(text, query):
    sentences = [elem.text for elem in list(sentenize(text))]
    docs = sentences + [query]
    modified_docs = [[stemmer.stem(i.lower()) for i in tokenize(d.translate(str.maketrans('','',string.punctuation))) if i.lower() not in stop_words] for d in docs]
    modified_docs = [' '.join(i) for i in modified_docs]
    tf_idf = TfidfVectorizer().fit_transform(modified_docs)

    minimum = (1, None)
    for i in range(len(sentences)):
        if cosine(tf_idf[i].todense(), tf_idf[len(sentences)].todense()) < minimum[0]:
            minimum = (cosine(tf_idf[i].todense(), tf_idf[len(sentences)].todense()), i)
    if minimum[1] == None:
        return None
    return sentences[minimum[1]]


In [217]:
input_filename = "dataset_281937_1-13.txt"
test = []
with open(input_filename, encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter='\t')
    isfirst = True
    sgere = 0
    for row in tqdm(csv_reader):
        print(sgere, end = ' ')
        sgere += 1
        if isfirst:
            isfirst = False
            continue
        question = row[3]
        id_ = row[1]
        if question[-1] == '?':
            question = question[:-1]
        sentence = find_closest_sentence(row[2], question)
        if sentence == None:
            sentence = row[2]
        sentence = re.split('(\W)', sentence)
        question = re.split('(\W)', question)
        test.append((sentence,
                     question,
                     id_))


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [218]:
stemmed_test = [stem_words(item[0], item[1]) for item in tqdm(test)]
stemmed_padded_test = pad_sequence(stemmed_test, batch_first=True, padding_value=pad_value)
indices_test = [get_words_indices(item[0]) for item in tqdm(test)]

In [0]:
with torch.no_grad():
    test_loader = torch.utils.data.DataLoader(stemmed_padded_test, batch_size=batch_size)
    ans = None
    
    for batch_data in test_loader:
        x = batch_data.to(device)
        output = model(x.long())
        _, ansx = output.max(dim=2)
        ansx = ansx.cpu().numpy()
        if ans is None:
            ans = ansx
        else:
            ans = np.append(ans, ansx, axis=0)

In [0]:
output_filename = "output.txt"

with open(output_filename, "w", encoding="utf-8") as output_file:
    for tags, row in zip(ans, test):
        start, end = tags
        if start > end:
            start, end = end, start
        if end >= len(row[0]):
            start, end = 0, len(row[0]) - 1
        output_file.write(row[2] + "\t" + "".join(row[0][start:end + 1]) + "\n")
